In [1]:
import json
import requests
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Make MBTA API request

In [4]:
CLIENT_KEY = '26e129e96c2249cc8478881689ee208c'
#     url = "https://api-v3.mbta.com/shapes?page%5Boffset%5D={}&page%5Blimit%5D={}&filter%5Bactivity%5D=BOARD%2CEXIT%2CRIDE&api_key={}".format(offset, page, CLIENT_KEY)
# &filter[route]=0%2C1
#page%5Boffset%5D={}&page%5Blimit%5D={}&

def fetch_mbta_v3(url):
    fetched = []

    offset = 0
    page = 20
    total = 200 # over 10000000 rows

    while offset <= total - page:
        url = f"https://api-v3.mbta.com/{url}?page%5Boffset%5D={page}&page%5Blimit%5D={offset}&include=parent_station&api_key={CLIENT_KEY}"
        print(url)
        try:
            results = requests.get(url).json()
            print('Fetched page {} with offset {}'.format(page, offset))
            print('results', results)

            fetched += [d['attributes'] for d in results['data']]
            offset += page
        except json.decoder.JSONDecodeError as err:
            print('API error: {}'.format(err))
            offset = total
    
    return fetched

In [5]:
stops = fetch_mbta_v3("stops")

https://api-v3.mbta.com/stops?page%5Boffset%5D=20&page%5Blimit%5D=0&include=parent_station&api_key=26e129e96c2249cc8478881689ee208c


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



KeyError: 'data'

In [6]:
stops

[{'address': None,
  'at_street': None,
  'description': None,
  'latitude': 42.489729,
  'location_type': 0,
  'longitude': -70.968996,
  'municipality': 'Lynn',
  'name': 'Lynnfield St @ St Marys Cemetery',
  'on_street': 'Lynnfield Street',
  'platform_code': None,
  'platform_name': None,
  'vehicle_type': 3,
  'wheelchair_boarding': 0},
 {'address': None,
  'at_street': 'Saint Richard Street',
  'description': None,
  'latitude': 42.323534,
  'location_type': 0,
  'longitude': -71.084009,
  'municipality': 'Boston',
  'name': 'Walnut Ave @ St Richard St',
  'on_street': 'Walnut Avenue',
  'platform_code': None,
  'platform_name': None,
  'vehicle_type': 3,
  'wheelchair_boarding': 0},
 {'address': None,
  'at_street': 'Page Road',
  'description': None,
  'latitude': 42.353395,
  'location_type': 0,
  'longitude': -71.208103,
  'municipality': 'Newton',
  'name': 'Walnut St @ Page Rd',
  'on_street': 'Walnut Street',
  'platform_code': None,
  'platform_name': None,
  'vehicle_typ

In [7]:
df_stops = pd.DataFrame(data=stops, columns=['name', 'latitude', 'longitude', 'municipality', 'parent_station'])
df_stops.head()

,name,latitude,longitude,municipality,parent_station
0,Lynnfield St @ St Marys Cemetery,42.489729,-70.968996,Lynn,NaN
1,Walnut Ave @ St Richard St,42.323534,-71.084009,Boston,NaN
2,Walnut St @ Page Rd,42.353395,-71.208103,Newton,NaN
3,Forest Hills,NaN,NaN,Boston,NaN
4,Main St @ Banks Pl,42.443302,-71.068052,Melrose,NaN


In [9]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9867 entries, 0 to 9866
Data columns (total 5 columns):
name              9867 non-null object
latitude          8504 non-null float64
longitude         8504 non-null float64
municipality      9867 non-null object
parent_station    0 non-null float64
dtypes: float64(3), object(2)
memory usage: 385.6+ KB


In [10]:
df_stops[df_stops.duplicated()]

,name,latitude,longitude,municipality,parent_station
69,Sullivan Square,NaN,NaN,Boston,NaN
91,Arlington,NaN,NaN,Boston,NaN
114,North Quincy,NaN,NaN,Quincy,NaN
146,Maverick,NaN,NaN,Boston,NaN
162,Tufts Medical Center,NaN,NaN,Boston,NaN
...,...,...,...,...,...
9850,Harvard,NaN,NaN,Cambridge,NaN
9852,Courthouse,NaN,NaN,Boston,NaN
9854,Wellesley Square,42.297526,-71.294173,Wellesley,NaN
9864,Courthouse,NaN,NaN,Boston,NaN


In [ ]:
df_stops.name.value_counts()

In [ ]:
# The stop may be inside a station.
# If /data/{index}/relationships/parent_station/data/id is present, 
# you should look up the parent station (/stops/{parent_id}) and use its location 
# to give direction first to the parent station and then route from there to the stop.

# dupes bc stations represent multiple lines